# Droplet Classification Pipeline

This notebook implements an automated pipeline for detecting and classifying droplets in microscopy images using a pre-trained XGBoost classifier.

## Pipeline Overview
1. **Quadrant Extraction**: Split multi-channel microscopy images into separate quadrants
2. **Droplet Detection**: Load pre-computed bounding boxes for detected droplets
3. **Feature Extraction**: Extract deep features from droplet images using MobileNetV2
4. **Classification**: Classify droplets using pre-trained XGBoost model
5. **Analysis**: Generate annotated images and combined spreadsheets with classification results

## Input Requirements
- Multi-channel microscopy TIF images organized in directories
- Pre-computed droplet bounding boxes (`.pkl` files)
- Pre-trained XGBoost classifier model
- CSV metadata files for each experiment

## Import Dependencies

In [ ]:
# Standard library imports
import os
import re
import pickle as pkl
from pathlib import Path
from collections import defaultdict
import random

# Data processing and analysis
import numpy as np
import pandas as pd
from PIL import Image
import cv2

# Machine learning
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb

# Deep learning
import torch
from torchvision import models
from torchvision.transforms import v2 as transforms

# Progress tracking
from tqdm import tqdm

# Custom data processing utilities
from src.data_processing import *

## Helper Functions

Functions for droplet detection, classification, and analysis.

In [ ]:
def _get_precomputed_droplets_worker(cur_quadrant_dir, quadrant_name):
    """
    Load pre-computed droplet bounding boxes and create detection visualizations.
    
    This function processes a single quadrant directory, loading droplet detection
    bounding boxes from pickle files and creating annotated images showing detected droplets.
    
    Args:
        cur_quadrant_dir (str): Path to the quadrant directory containing TIF images
        quadrant_name (str): Name of the quadrant (e.g., "488", "546", "647")
    """
    # Skip if already processed
    if Path(f"{cur_quadrant_dir}/droplet_detection_pkl/").exists():
        return
    
    # Extract quadrant number from directory path
    m = re.search("quadrant_(\d)", cur_quadrant_dir)
    if m is None:
        raise Exception(f"Invalid quadrant directory: {cur_quadrant_dir}")

    # Load all bounding boxes for this quadrant
    droplets_dict = defaultdict(list)
    for cur_dir, cur_fname in get_files_matching_regex(
        f"{cur_quadrant_dir}/../",
        f"(?m)^\d_([A-Z0-9]{{2}}_)?{quadrant_name}_droplet_boxes.pkl$",
    ):
        # Extract TIF ID from filename
        m = re.search("(?m)^(\d)_", cur_fname)
        tif_id = int(m.groups()[0])

        # Load bounding boxes
        with open(f"{cur_dir}/{cur_fname}", "rb") as fp:
            cur_bboxes = pkl.loads(fp.read())

        # Process each bounding box
        for cur_k, cur_v in cur_bboxes.items():
            # Extract frame ID from key
            m = re.search(f"(?m)_(\d+)_{quadrant_name}$", cur_k)
            if m is None:
                raise Exception(
                    f"Key in {cur_dir}/{cur_fname} does not match: {cur_k} ({tif_id}, {quadrant_name})"
                )
            frame_id = int(m.groups()[0])
            
            # Store droplet info (filter by aspect ratio to keep approximately circular droplets)
            for bbox_idx, cur_bbox in enumerate(cur_v):
                # Skip droplets with aspect ratio too far from 1 (non-circular)
                if abs(1 - (cur_bbox[2] / cur_bbox[3])) > 0.15:
                    continue
                    
                droplets_dict[(tif_id, frame_id)].append(
                    (
                        bbox_idx,
                        cur_bbox[0],  # x coordinate
                        cur_bbox[1],  # y coordinate
                        0.25 * (cur_bbox[2] + cur_bbox[3]),  # average radius
                    )
                )

    # Get all TIF images in the quadrant directory
    tifs = list(
        sorted(get_files_matching_regex(cur_quadrant_dir, "(?m)^\d+_\d+\.tif$"))
    )

    # Process each TIF image
    for root, fname in tqdm(tifs, desc=f"Processing {quadrant_name}"):
        # Create output directories
        Path(f"{cur_quadrant_dir}/droplet_detection_pkl/").mkdir(exist_ok=True)
        Path(f"{cur_quadrant_dir}/droplet_detection_im/").mkdir(exist_ok=True)

        out_pkl_pth = f"{cur_quadrant_dir}/droplet_detection_pkl/{os.path.splitext(fname)[0]}_droplets.pkl"

        # Extract indices from filename
        m = re.search("(?m)^(\d+)_(\d+)\.tif$", fname)
        ex_idx = int(m.groups()[0])
        frame_idx = int(m.groups()[1])

        # Load image
        cur_fname = f"{root}/{fname}"
        cur_im = np.array(Image.open(cur_fname))

        # Get droplets for this image
        droplets = droplets_dict[(ex_idx, frame_idx)]
        circles = [((x, y), r) for _, x, y, r in droplets]

        # Save droplet data
        with open(out_pkl_pth, "wb") as fp:
            fp.write(pkl.dumps(droplets))
        
        # Create visualization with circles drawn on image
        cur_im = normalize_image(cur_im, (0.1, 99.9)).astype(np.uint8)
        cur_im_circles = draw_circles(circles, cur_im)

        Image.fromarray(cur_im_circles).save(
            f"{cur_quadrant_dir}/droplet_detection_im/{os.path.splitext(fname)[0]}_droplets.tif"
        )


def write_droplets(quadrant_dirs, subsample):
    """
    Extract and save individual droplet images for training dataset generation.
    
    Args:
        quadrant_dirs (list): List of (quadrant_dir, quadrant_name) tuples
        subsample (int): Subsampling factor (1 = save all droplets, 2 = save every 2nd, etc.)
    """
    for cur_quadrant_dir, _ in quadrant_dirs:
        print(f"Processing {cur_quadrant_dir}")
        dir_hash = hex(hash(cur_quadrant_dir))[-5:]

        # Skip if already processed
        if Path(f"{cur_quadrant_dir}/droplet_ims/").exists():
            continue
        Path(f"{cur_quadrant_dir}/droplet_ims/").mkdir()

        tifs = list(
            sorted(get_files_matching_regex(cur_quadrant_dir, "(?m)^\d+_\d+\.tif$"))
        )

        droplet_cnt = -1
        for root, fname in tqdm(tifs):
            pkl_pth = f"{cur_quadrant_dir}/droplet_detection_pkl/{os.path.splitext(fname)[0]}_droplets.pkl"

            if not Path(pkl_pth).exists():
                continue

            # Load droplet locations
            with open(pkl_pth, "rb") as fp:
                circles = pkl.loads(fp.read())

            # Load image
            cur_fname = f"{root}/{fname}"
            cur_im = np.array(Image.open(cur_fname))
            cur_im = normalize_image(cur_im, (0.1, 99.9))

            # Extract each droplet as a separate image
            for idx, x, y, r in circles:
                x, y, r = int(x), int(y), int(r)
                
                # Create circular mask
                mask = np.zeros_like(cur_im)
                cv2.circle(mask, (x, y), r, (1,), -1)
                
                # Apply mask and crop
                crop = cv2.multiply(cur_im, mask)
                crop = crop[y - r : y + r, x - r : x + r]

                droplet_cnt += 1
                
                # Apply subsampling
                if droplet_cnt % subsample != 0:
                    continue
                    
                # Save droplet image
                try:
                    Image.fromarray(crop).save(
                        f"{cur_quadrant_dir}/droplet_ims/{dir_hash}_{droplet_cnt}.png"
                    )
                except SystemError:
                    pass  # Skip if image saving fails


def classify_droplets(quadrant_dirs, classifier):
    """
    Classify all detected droplets using the pre-trained XGBoost model.
    
    This function extracts features from droplet images using MobileNetV2 and
    classifies them using the provided XGBoost classifier.
    
    Args:
        quadrant_dirs (list): List of (quadrant_dir, quadrant_name) tuples
        classifier (XGBWrapper): Pre-trained XGBoost classifier instance
    """
    for cur_quadrant_dir, _ in quadrant_dirs:
        # Skip if already processed
        if Path(f"{cur_quadrant_dir}/droplet_classification_pkl/").exists():
            continue
            
        print(f"Working on {cur_quadrant_dir}")
        Path(f"{cur_quadrant_dir}/droplet_classification_pkl/").mkdir(exist_ok=True)

        # Get all TIF images
        tifs = list(get_files_matching_regex(cur_quadrant_dir, "(?m)^\d+_\d+\.tif$"))
        
        # Collect all droplet images for batch processing
        cur_im_droplets = []
        droplet_indices = []
        circles_retained = []
        droplet_fnames = []
        
        for root, fname in tqdm(tifs, desc="Loading droplets"):
            pkl_pth = f"{cur_quadrant_dir}/droplet_detection_pkl/{os.path.splitext(fname)[0]}_droplets.pkl"
            if not Path(pkl_pth).exists():
                continue

            # Load droplet locations
            with open(pkl_pth, "rb") as fp:
                circles = pkl.loads(fp.read())

            # Load image
            cur_fname = f"{root}/{fname}"
            cur_im = np.array(Image.open(cur_fname))
            cur_im = normalize_image(cur_im, (0.1, 99.9))

            # Extract each droplet
            for idx, x, y, r in circles:
                x, y, r = int(x), int(y), int(r)
                
                # Create circular mask
                mask = np.zeros_like(cur_im)
                cv2.circle(mask, (x, y), r, (1,), -1)
                
                # Apply mask and crop
                crop = cv2.multiply(cur_im, mask)
                crop = crop[y - r : y + r, x - r : x + r]

                # Skip droplets that intersect image borders (resulting in empty crops)
                if 0 in crop.shape:
                    continue

                # Convert to RGB for feature extraction
                crop = Image.fromarray(crop).convert("RGB")
                cur_im_droplets.append(crop)
                droplet_indices.append(idx)
                circles_retained.append((idx, x, y, r))
                droplet_fnames.append(fname)

        if len(cur_im_droplets) == 0:
            continue

        # Classify droplets in batches
        # Larger batch size for better GPU utilization (adjust based on GPU memory)
        bs = 500 if torch.cuda.is_available() else 100
        length = len(cur_im_droplets)
        pred = []
        
        for batch_off in tqdm(range(0, length, bs), desc="Classifying"):
            this_batch = cur_im_droplets[batch_off : batch_off + bs]
            
            # Extract features using MobileNetV2
            this_batch_feats = get_features_batched(this_batch)

            # Classify using XGBoost
            this_batch_pred = classifier.predict(this_batch_feats)
            pred.append(this_batch_pred)

        # Combine predictions
        preds = np.concatenate(pred)
        
        # Organize predictions by filename
        out_dict = {xx: ([], []) for xx in set(droplet_fnames)}
        for idx in range(length):
            cur_fname = droplet_fnames[idx]
            cur_droplet_idx = droplet_indices[idx]
            cur_pred = preds[idx]

            out_dict[cur_fname][0].append(cur_droplet_idx)
            out_dict[cur_fname][1].append(cur_pred)

        # Save predictions for each file
        for cur_fname, (droplet_indices, cur_preds) in out_dict.items():
            out_pkl_pth = f"{cur_quadrant_dir}/droplet_classification_pkl/{os.path.splitext(cur_fname)[0]}_classified.pkl"

            with open(out_pkl_pth, "wb") as fp:
                fp.write(pkl.dumps(list(zip(droplet_indices, cur_preds))))


def create_combined_spreadsheet(quadrant_dirs):
    """
    Generate combined analysis spreadsheet with classification results.
    
    Merges classification predictions with existing CSV metadata to create
    a comprehensive analysis spreadsheet.
    
    Args:
        quadrant_dirs (list): List of (quadrant_dir, quadrant_name) tuples
    """
    for cur_quadrant_dir, _ in quadrant_dirs:
        # Skip if already processed
        if Path(f"{cur_quadrant_dir}/../analysis.csv").exists():
            continue

        # Find and load existing CSV file
        df = None
        for root, fname in get_files_matching_regex(
            f"{cur_quadrant_dir}/../", "(?m)^.*\.csv$"
        ):
            print(f"Loading metadata from {root}/{fname}")
            df = pd.read_csv(f"{root}/{fname}")
            break  # Use first CSV found

        if df is None:
            print(f"No CSV file found for {cur_quadrant_dir}, skipping")
            continue

        # Load all classification results
        droplets = {}
        tifs = list(get_files_matching_regex(cur_quadrant_dir, "(?m)^\d+_\d+\.tif$"))
        
        for root, fname in tqdm(tifs, desc="Loading classifications"):
            # Parse filename
            m = re.search("(?m)^(\d+)_(\d+)\.tif$", fname)
            im_idx = int(m.groups()[0])
            frame_idx = int(m.groups()[1])

            # Load classification results
            class_pkl_pth = f"{cur_quadrant_dir}/droplet_classification_pkl/{os.path.splitext(fname)[0]}_classified.pkl"
            try:
                with open(class_pkl_pth, "rb") as fp:
                    classified = pkl.loads(fp.read())
            except FileNotFoundError:
                continue
                
            # Store classifications by (tif_id, frame_id, droplet_id)
            for droplet_id, cur_class in classified:
                droplets[(im_idx, frame_idx, droplet_id)] = cur_class

        # Add classification column to dataframe
        df["class"] = 0
        for idx, row in df.iterrows():
            try:
                # Parse TIF ID (handle both numeric and string formats)
                try:
                    tif_id = int(row["tif"])
                except ValueError:
                    m = re.search("(\d+)_[A-Z]+", row["tif"])
                    tif_id = int(m.groups()[0])
                
                # Look up classification
                cur_class = droplets[
                    (tif_id, int(row["frame"]), int(row["droplet_id"]))
                ]
                df.at[idx, "class"] = cur_class
            except KeyError:
                # Some droplets were skipped (e.g., intersecting image boundaries)
                continue

        # Save updated spreadsheet
        output_path = f"{cur_quadrant_dir}/../analysis.csv"
        df.to_csv(output_path, sep=",", index=False)
        print(f"Saved analysis to {output_path}")


def write_droplet_debug_ims(quadrant_dir):
    """
    Create annotated images showing classified droplets with labels.
    
    Generates visualization images with droplets circled and labeled with their
    classification results for visual verification.
    
    Args:
        quadrant_dir (str): Path to the quadrant directory
    """
    # Skip if already processed
    if Path(f"{quadrant_dir}/droplet_classification_im/").exists():
        return
    Path(f"{quadrant_dir}/droplet_classification_im/").mkdir(exist_ok=True)

    tifs = list(sorted(get_files_matching_regex(quadrant_dir, "(?m)^\d+_\d+\.tif$")))
    
    for root, fname in tqdm(tifs, desc="Creating debug images"):
        detection_pkl_pth = f"{quadrant_dir}/droplet_detection_pkl/{os.path.splitext(fname)[0]}_droplets.pkl"
        classification_pkl_pth = f"{quadrant_dir}/droplet_classification_pkl/{os.path.splitext(fname)[0]}_classified.pkl"

        # Load detection and classification results
        with open(detection_pkl_pth, "rb") as fp:
            dets = pkl.loads(fp.read())
        
        try:
            with open(classification_pkl_pth, "rb") as fp:
                classes = pkl.loads(fp.read())
        except FileNotFoundError:
            continue

        # Load image
        cur_fname = f"{root}/{fname}"
        cur_im = np.array(Image.open(cur_fname))

        # Match detections with classifications
        classes = {idx: cls for idx, cls in classes}
        retained_dets = []
        retained_cls = []
        
        for cur in dets:
            if cur[0] in classes:
                retained_dets.append(cur)
                retained_cls.append((cur[0], classes[cur[0]]))

        # Draw circles with labels
        cur_im = normalize_image(cur_im, (0.1, 99.9)).astype(np.uint8)
        cur_im_circles = draw_circles(
            [((x, y), r) for _, x, y, r in retained_dets],
            cur_im,
            labels=[f"{yy[0:3]} ({xx})" for xx, yy in retained_cls],  # Label: "Class (ID)"
        )
        
        # Save annotated image
        Image.fromarray(cur_im_circles).save(
            f"{quadrant_dir}/droplet_classification_im/{os.path.splitext(fname)[0]}_classification.tif"
        )


# Global model cache to avoid reloading
_FEATURE_EXTRACTOR_MODEL = None
_FEATURE_EXTRACTOR_DEVICE = None

def get_feature_extractor_model():
    """
    Get or create the cached feature extraction model.
    This avoids reloading the model for every batch, providing significant speedup.
    
    Returns:
        tuple: (model, device)
    """
    global _FEATURE_EXTRACTOR_MODEL, _FEATURE_EXTRACTOR_DEVICE
    
    if _FEATURE_EXTRACTOR_MODEL is None:
        print("Loading MobileNetV2 feature extractor (one-time initialization)...")
        _FEATURE_EXTRACTOR_DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        _FEATURE_EXTRACTOR_MODEL = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.DEFAULT).to(_FEATURE_EXTRACTOR_DEVICE)
        _FEATURE_EXTRACTOR_MODEL.eval()
        
        # Remove classification layer to extract features from penultimate layer
        _FEATURE_EXTRACTOR_MODEL.classifier[1] = torch.nn.Identity()
        
        print(f"Model loaded on device: {_FEATURE_EXTRACTOR_DEVICE}")
    
    return _FEATURE_EXTRACTOR_MODEL, _FEATURE_EXTRACTOR_DEVICE


def extract_features(image, is_batch=False):
    """
    Extract deep features from images using MobileNetV2 pre-trained on ImageNet.
    Uses a cached model for improved performance.
    
    Args:
        image (torch.Tensor): Input image tensor(s)
        is_batch (bool): Whether input is a batch of images. Defaults to False.
    
    Returns:
        numpy.ndarray: Extracted feature vectors (1280-dimensional for MobileNetV2)
    """
    # Get cached model
    model, device = get_feature_extractor_model()
    
    # Add batch dimension if single image
    if is_batch:
        batch_img_tensor = image
    else:
        batch_img_tensor = image.unsqueeze(0)

    # Move to GPU if available
    batch_img_tensor = batch_img_tensor.to(device)

    # Extract features without gradient computation
    with torch.no_grad():
        features = model(batch_img_tensor)

    # Flatten and convert to numpy
    if is_batch:
        features_flattened = torch.flatten(features, start_dim=1).detach().cpu().numpy()
    else:
        features_flattened = torch.flatten(features, start_dim=1).detach().cpu().numpy()[0]

    return features_flattened


def get_features_batched(images, adjust_size=True):
    """
    Extract features from a batch of images efficiently.
    
    Args:
        images (list): List of PIL Images
        adjust_size (bool): Whether to apply preprocessing transforms. Defaults to True.
    
    Returns:
        numpy.ndarray: Feature matrix of shape (n_images, 1280)
    """
    # Define standard ImageNet preprocessing
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToImage(),
        transforms.ToDtype(torch.float32, scale=True),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    # Apply preprocessing if needed
    if adjust_size:
        images = [preprocess(img) for img in images]

    # Stack into batch and extract features
    batch = torch.stack(images)
    features = extract_features(batch, is_batch=True)

    return features


class XGBWrapper:
    """
    Wrapper class for XGBoost to handle multi-class classification with label encoding.
    This class is required to unpickle the pre-trained model.
    """
    
    def __init__(self, params, epochs):
        """
        Initialize XGBoost wrapper.
        
        Args:
            params (dict): XGBoost parameters (max_depth, eta, objective, etc.)
            epochs (int): Number of boosting rounds
        """
        self.params = params
        self.epochs = epochs
        self.label_encoder = LabelEncoder()
        self.model = None

    def fit(self, X, y):
        """
        Train XGBoost model on feature matrix X with labels y.
        
        Args:
            X (numpy.ndarray): Feature matrix
            y (array-like): Class labels (strings)
        """
        # Encode string labels to integers
        labels = self.label_encoder.fit_transform(y)
        self.params["num_class"] = len(self.label_encoder.classes_)
        dtrain = xgb.DMatrix(X, label=labels)

        # Train the model
        self.model = xgb.train(self.params, dtrain, self.epochs)

    def predict(self, X):
        """
        Predict class labels for feature matrix X.
        
        Args:
            X (numpy.ndarray): Feature matrix
            
        Returns:
            numpy.ndarray: Predicted class labels (strings)
        """
        dtest = xgb.DMatrix(X)
        y_pred = self.model.predict(dtest)
        y_pred = y_pred.argmax(axis=1)
        y_pred = self.label_encoder.inverse_transform(y_pred)
        return y_pred

## Configuration and Pipeline Execution

In [ ]:
dataset_path = "/Users/klavs/Desktop/PhD/Tanu/image-thresholding/data/Corrected_exp1_50uMg3bp1pro_1.5mgml_lysg156e"
skip_droplet_classification = False  # Set to True to skip classification step
write_raw_droplets = False  # Set to True to export raw droplet images for training data
skip_debug_ims = False  # Set to True to skip creating annotated debug images
skip_combined_spreadsheet = False  # Set to True to skip generating analysis CSV

trained_n = 100000  # Number of samples used to train the classifier
feature = f"mobilenetv2_max{trained_n}"
classifier = "xgb"

# =============================================================================
# Quadrant Configuration
# =============================================================================
# Map experiment conditions to (classification_wavelength, reference_wavelength)
# Classification wavelength: channel containing droplets to classify
# Reference wavelength: channel used for droplet detection
classify_quadrant = {
    "G3BP1_2/P525L": (488, 647),  # Classify 546nm channel, detect in 647nm
}

# Quadrant position mapping (4-quadrant layout)
# Layout:
#   0  1
#   2  3
quadrant_map = {
    488: 3,  # Bottom right
    546: 2,  # Bottom left
    647: 1,  # Top right (reference channel)
}

# Quadrant names for file identification
quadrant_names = {
    1: "647",
    2: "546",
    3: "488",
}

# Convert wavelength pairs to quadrant indices
classify_quadrant = {
    experiment: (quadrant_map[classify_channel], quadrant_map[detect_channel])
    for experiment, (classify_channel, detect_channel) in classify_quadrant.items()
}

print(f"Quadrant configuration: {classify_quadrant}")
print("\nLoading classifier...")
classifier_path = f"{os.path.join(dataset_path, classifier+'_trained_'+feature)}.pkl"
print(f"Using classifier from: {classifier_path}")

with open(classifier_path, "rb") as fp:
    clf = pkl.loads(fp.read())

print(f"Classifier loaded successfully")

print("\n" + "="*80)
print("STEP 1: Extracting quadrant images from multi-channel TIFs")
print("="*80)
quadrant_dirs = list(extract_quadrants(dataset_path, classify_quadrant))
quadrant_dirs = [(xx, quadrant_names[yy]) for xx, yy in quadrant_dirs]
print(f"Found {len(quadrant_dirs)} quadrant directories to process")

print("\n" + "="*80)
print("STEP 2: Reading pre-computed droplet bounding boxes")
print("="*80)

# Process sequentially (multiprocessing doesn't work well in Jupyter notebooks)
for quadrant_dir, quadrant_name in tqdm(quadrant_dirs, desc="Processing quadrants"):
    _get_precomputed_droplets_worker(quadrant_dir, quadrant_name)

print("Droplet detection loading complete")

if write_raw_droplets:
    print("\n" + "="*80)
    print("STEP 3: Writing raw droplet images for training data generation")
    print("="*80)
    write_droplets(quadrant_dirs, subsample=1)
else:
    print("\nSkipping raw droplet export (write_raw_droplets=False)")

if skip_droplet_classification:
    print("\nSkipping droplet classification (skip_droplet_classification=True)")
else:
    print("\n" + "="*80)
    print("STEP 4: Classifying droplets using XGBoost")
    print("="*80)
    classify_droplets(quadrant_dirs, clf)
    print("Classification complete")

if skip_debug_ims:
    print("\nSkipping debug image generation (skip_debug_ims=True)")
else:
    print("\n" + "="*80)
    print("STEP 5: Creating annotated images with classification labels")
    print("="*80)
    # Process sequentially (multiprocessing doesn't work well in Jupyter notebooks)
    for quadrant_dir, _ in tqdm(quadrant_dirs, desc="Creating debug images"):
        write_droplet_debug_ims(quadrant_dir)
    print("Debug images created")

if skip_combined_spreadsheet:
    print("\nSkipping spreadsheet generation (skip_combined_spreadsheet=True)")
else:
    print("\n" + "="*80)
    print("STEP 6: Creating combined analysis spreadsheet")
    print("="*80)
    create_combined_spreadsheet(quadrant_dirs)
    print("Analysis spreadsheet generated")

print("\n" + "="*80)
print("PIPELINE COMPLETE")
print("="*80)